In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
#np.random.seed(1337)  # for reproducibility

import matplotlib.pyplot as plt
import random
import time

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers.convolutional import Convolution1D, MaxPooling1D

TdataSamples = 10000
FdataSamples = 10000
dataPoints = 100 #No. of data points, should be 73440 for 17 quarters
min_period = 2
max_period = int(dataPoints / 5)
testSetFraction = 0.2
trainSetSize =  (TdataSamples + FdataSamples) * (1 - testSetFraction)
testSetSize = (TdataSamples + FdataSamples) * (testSetFraction)


def gen_Tsignal(dataPoints, min_period, max_period):
        signal = []
        period = np.random.randint(min_period,max_period)
    
        for i in range(dataPoints):
            if( ( i % period) == 0):
                signal.append(1)
            else:
                signal.append(0)
        
        #signal = np.array(signal)
        
        
        if(signal[-1] == 0):
            signal = np.roll(signal, 1)
            if(signal[-1] == 0):
                signal = np.roll(signal, 1)
                if(signal[-1] == 0):
                    signal = np.roll(signal, 1)
            
        else:
            signal = np.roll(signal, 0)

        #print(signal)
        return(signal)


def gen_Fsignal(dataPoints):
        signal = []
    
        for i in range(dataPoints):
            rflag = np.random.randint(0, 2)
            if( rflag == 1):
                signal.append(1)
            else:
                signal.append(0)
        
        signal = np.array(signal)
        return(signal)    



def gen_Tdata(TdataSamples, dataPoints, min_period, max_period):
        
        TdatasetList = []
        for i in range(TdataSamples):
            signal = []
            signal = gen_Tsignal(dataPoints, min_period, max_period)
            #print(signal)
            TdatasetList.append(signal)
        
        Tdataset = np.array(TdatasetList)
        return(Tdataset)


Tdataset = gen_Tdata(TdataSamples, dataPoints, min_period, max_period)

X_True = Tdataset
y_True = np.ones(TdataSamples).tolist()
#print(X_True)
#print(y_True)
print("True dataset made")


def gen_Fdata(FdataSamples, dataPoints):
        FdatasetList = []
        for i in range(FdataSamples):
            signal = []
            signal = gen_Fsignal(dataPoints)
            #print(signal)
            FdatasetList.append(signal)
        
        Fdataset = np.array(FdatasetList)
        return(Fdataset)
    
    
Fdataset = gen_Fdata(FdataSamples, dataPoints)

X_False = Fdataset
y_False = np.zeros(FdataSamples).tolist()
#print(X_False)
#print(y_False)
print("False dataset made")

X = np.concatenate( (X_True, X_False) , axis = 0)
y = np.concatenate( (y_True, y_False) , axis = 0)

X = np.array(X)
y = np.array(y)


testIdx = random.sample(range(0, len(X)), int(testSetFraction * len(X) ))
trainIdx = list( set(range(0, len(X))) - set(testIdx) )
random.shuffle(trainIdx)

X_train = [X[i] for i in trainIdx]
X_test = [X[i] for i in testIdx]

y_train = [y[i] for i in trainIdx]
y_test = [y[i] for i in testIdx]

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

#print(X_train)
#print(y_train)

#print(X_test)
#print(y_test)

X_train = X_train.reshape((trainSetSize,dataPoints,1))
X_test = X_test.reshape((testSetSize,dataPoints,1))

y_train = y_train.reshape((trainSetSize,1))
y_test = y_test.reshape((testSetSize,1))

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

time.sleep(1)
print("Test and Train sets made....")

input_size = X_train.shape[1]
output_size = y_train.shape[1]
hidden_size = 4
nb_epoch = 1
batch_size = 16

time.sleep(1)
print('Build model...')
model = Sequential()

model.add(LSTM(output_dim=hidden_size, input_dim=1, return_sequences = False))

#model.add(GRU(output_dim=64, input_dim=1))
#model.add(Dropout(0.25))
#model.add(Activation('relu'))
#two FC layers
#model.add(Dense(128))
#model.add(Activation('relu'))
#model.add(Dropout(0.25))
#model.add(Dense(128))
#model.add(Dropout(0.25))
#model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(4))
model.add(Dense(1)) 
#model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              class_mode='binary')

print('Train...')
model.fit(X_train, y_train, batch_size=batch_size,
          nb_epoch=nb_epoch, show_accuracy=True,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size,
                            show_accuracy=True)
print('Test score:', score)
print('Test accuracy:', acc)


True dataset made
False dataset made
(16000, 100, 1)
(4000, 100, 1)
(16000, 1)
(4000, 1)
Test and Train sets made....
Build model...